# Identify Flares/Stars with coinciding SPT and TESS observations
This notebook searches TESS for the flaring stars in the SPT flare catalog (Tandoi et.al. 24) and identifies sectors of coinciding observation.
The identifying data of the stars with coinciding observations are saved to `spt_tess_candidates.pkl`.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import pandas as pd

from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from scipy.linalg import lstsq
from scipy.optimize import curve_fit

import lightkurve as lk
import tessreduce as tr

import pickle

<span style="color:red; font-weight:bold;"> See these tools for relevant tess sector identification:</span> 
- TESS-point: https://github.com/tessgi/tess-point
  - This would greatly restrict sectors we search for each star, so worth exploring for very large catalogs
- TESS-point Web Tool: https://heasarc.gsfc.nasa.gov/wsgi-scripts/TESS/TESS-point_Web_Tool/TESS-point_Web_Tool/wtv_v2.0.py/

First we retrieve the flare locations and times for SPT and TESS orbit details

The SPT "Flare Star Catalog" is retrieved from https://pole.uchicago.edu/public/data/tandoi24/#Overview.

TESS sector orbit times are retrieved from https://tess.mit.edu/observations/.

In [11]:
spt_flares_df = pd.read_csv('../data/SPT/2023_june2025_winter_stars_for_TESS.csv')
spt_flares_df = spt_flares_df.replace()
spt_flares_df['mjd'] = spt_flares_df['start_time'].apply(lambda x: Time(x, scale= 'utc', format='iso').utc.mjd)
spt_flares_df.sort_values(by= 'mjd')
spt_flares_df['sectors'] = [[] for _ in range(len(spt_flares_df))]
print(spt_flares_df.columns)
spt_flares_df = spt_flares_df.rename(columns={'source_ra': 'ra', 'source_dec': 'dec'})

TESS_sectors_df = pd.read_csv("..\data\TESS\TESS_orbit_times.csv")
# Times are stored by orbit, so first merge by sector
TESS_sectors_df = TESS_sectors_df.groupby("Sector").agg({
    "Start of Orbit": "min",
    "End of Orbit": "max"
}).reset_index()
TESS_sectors_df.columns = ["Sector", "Sector Start", "Sector End"]

# Formatting to MJD for comparison
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df['Sector'] = TESS_sectors_df['Sector'].apply(lambda x: int(x))
TESS_sectors_df['Sector Start'] = TESS_sectors_df['Sector Start'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
TESS_sectors_df['Sector End'] = TESS_sectors_df['Sector End'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
print(TESS_sectors_df.columns)

Index(['peak_obsid', 'DR3_source_id', 'source_ra', 'source_dec', 'start_time',
       'end_time', 'mjd', 'sectors'],
      dtype='object')
Index(['Sector', 'Sector Start', 'Sector End'], dtype='object')


Using this data we find the range of TESS sectors over which SPT observed the flares. This is not precise and simply cuts out sectors before the first flare and after the last flare in the catalog.

In [12]:
spt_t_bounds = (spt_flares_df.iloc[0]['mjd'], spt_flares_df.iloc[-1]['mjd'])
t_bounds = Time(spt_t_bounds, format= 'mjd')
print(t_bounds.iso)

# Look for TESS Sectors between first flare start and last flare end
valid_sectors = TESS_sectors_df.loc[(TESS_sectors_df['Sector Start'] <= t_bounds[1].value) & (TESS_sectors_df['Sector End'] >= t_bounds[0].value)]
valid_sectors = set(valid_sectors['Sector'].apply(lambda x: int(x)))
print(valid_sectors)

['2023-03-31 09:24:59.000' '2025-06-04 11:07:06.000']
{63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93}


For each flare in spt_flare_catalog first grab FFIs for all sectors that contain its skycoordinates using tesscut. Lightkurve search is generally fast and I suspect may require less local runtime than checking if flare falls withing explicit time range of each sector.

Then, for the sectors where TESS is looking at the star and check the sector times to identify the one with the flare.

In [13]:
for i, flare in spt_flares_df.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    flare_obs_start = Time(flare['mjd'],  format='mjd', scale='utc')
    c = SkyCoord(ra, dec, unit= 'deg')

    # Search for any sectors where TESS is looking at the star
    possible_matches = lk.search_tesscut(c, [sector for sector in valid_sectors])
    if len(possible_matches) >= 1:
        # Check time range for each sector
        for data_prod in possible_matches:
            sector = int(data_prod.mission[0][11:])
            sector_time = TESS_sectors_df[TESS_sectors_df['Sector'] == sector]
            sector_start = sector_time['Sector Start'].iloc[0]
            sector_end = sector_time['Sector End'].iloc[0]
            if sector_start < flare_obs_start.value < sector_end:
                print(f'{i}/{spt_flares_df.shape[0]}: Match found for Sector {sector}')
                flare['sectors'] = flare['sectors'].append(sector)
            else:
                print(f'{i}/{spt_flares_df.shape[0]}: Sector {sector} - no overlap in time')
    else:
        print(f'{i}/{spt_flares_df.shape[0]}: no nearby targets')


0/38: Sector 67 - no overlap in time
1/38: Sector 68 - no overlap in time
2/38: Sector 67 - no overlap in time
3/38: Sector 68 - no overlap in time
4/38: Sector 69 - no overlap in time
5/38: Sector 68 - no overlap in time
6/38: Sector 69 - no overlap in time
7/38: Sector 67 - no overlap in time
8/38: Sector 69 - no overlap in time
9/38: Sector 69 - no overlap in time


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (39.95198541, -42.88485681)>".


10/38: no nearby targets
11/38: Sector 69 - no overlap in time
12/38: Match found for Sector 67
13/38: Sector 68 - no overlap in time
13/38: Match found for Sector 67
14/38: Match found for Sector 67
15/38: Sector 67 - no overlap in time
16/38: Sector 67 - no overlap in time
17/38: Sector 68 - no overlap in time
17/38: Match found for Sector 69


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (48.10760539, -44.42078289)>".


18/38: no nearby targets
19/38: Sector 68 - no overlap in time
20/38: Sector 67 - no overlap in time
21/38: Sector 68 - no overlap in time
21/38: Sector 69 - no overlap in time
22/38: Sector 68 - no overlap in time
22/38: Sector 69 - no overlap in time
23/38: Sector 69 - no overlap in time
24/38: Sector 67 - no overlap in time
25/38: Sector 68 - no overlap in time
25/38: Sector 67 - no overlap in time
26/38: Sector 69 - no overlap in time
27/38: Sector 68 - no overlap in time
27/38: Sector 69 - no overlap in time
28/38: Sector 67 - no overlap in time
29/38: Sector 67 - no overlap in time
30/38: Sector 67 - no overlap in time
31/38: Sector 69 - no overlap in time
32/38: Sector 69 - no overlap in time
33/38: Sector 69 - no overlap in time
34/38: Sector 68 - no overlap in time
34/38: Sector 67 - no overlap in time


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (42.7979808, -47.88549198)>".


35/38: no nearby targets
36/38: Sector 69 - no overlap in time
37/38: Sector 68 - no overlap in time
37/38: Sector 69 - no overlap in time


In [14]:
spt_tess_candidates = spt_flares_df[spt_flares_df['sectors'].apply(lambda x: len(x) > 0)].reset_index()
spt_tess_candidates

,index,peak_obsid,DR3_source_id,ra,dec,start_time,end_time,mjd,sectors
0,12,205545046,6463748969563201408,319.964017,-53.449742,2023-07-07 23:50:47,2023-07-08 01:58:35,60132.993600,[67]
1,13,206191850,6388014157668558080,352.241030,-68.043088,2023-07-15 11:30:51,2023-07-15 13:38:39,60140.479757,[67]
2,14,206333946,6463546350186454144,320.183645,-54.633106,2023-07-17 02:59:07,2023-07-17 05:06:55,60142.124387,[67]
3,17,210122640,4904142818493218304,13.575856,-59.422393,2023-08-29 23:24:01,2023-08-30 01:31:49,60185.975012,[69]


Print out flare skycoord in hmsdms to search in SINBAD and find common identifier for flares; I chose TIC.

In [15]:
for i, flare in spt_tess_candidates.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    c = SkyCoord(ra, dec, frame= 'icrs', unit= 'deg')
    print(i, c.to_string('hmsdms'))

0 21h19m51.36413543s -53d26m59.0716511s
1 23h28m57.84720912s -68d02m35.11602777s
2 21h20m44.07488332s -54d37m59.1800009s
3 00h54m18.20546204s -59d25m20.61364507s


Add TIC to dataframe

In [18]:
spt_tess_candidates['TIC'] = spt_tess_candidates['peak_obsid']

spt_tess_candidates.at[0,'TIC'] = "TIC79394645" #79394646
spt_tess_candidates.at[1,'TIC'] = "TIC229807000"
spt_tess_candidates.at[2,'TIC'] = "TIC219317273"
spt_tess_candidates.at[3,'TIC'] = None


spt_tess_candidates[['peak_obsid', 'DR3_source_id', 'TIC', 'sectors']]

,peak_obsid,DR3_source_id,TIC,sectors
0,205545046,6463748969563201408,TIC79394645,[67]
1,206191850,6388014157668558080,TIC229807000,[67]
2,206333946,6463546350186454144,TIC219317273,[67]
3,210122640,4904142818493218304,None,[69]


Save dataframe

In [19]:
spt_tess_candidates.to_pickle('..\data\spt_tess_candidates_23_jun25.pkl')